In [1]:
import gc
gc.enable()

In [2]:
!pip install git+https://github.com/qubvel/segmentation_models

  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-6iqvgpc3
    100% |████████████████████████████████| 81kB 5.4MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-y9_1wj61/wheels/49/cf/46/cbb4bb64518c402aea99df9d466f1081450597e653256bbcf4
Successfully built segmentation-models
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from keras import backend as K
from keras.models import load_model
from segmentation_models.losses import bce_jaccard_loss
import os
import numpy as np
from skimage.io import imread
import pandas as pd

def f2_micro(y_true, y_pred):
    agreement = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    total_true_positive = K.sum(K.round(K.clip(y_true, 0, 1)))
    total_pred_positive = K.sum(K.round(K.clip(y_pred, 0, 1)))
    recall = agreement / (total_true_positive + K.epsilon())
    precision = agreement / (total_pred_positive + K.epsilon())
    return (1+2**2)*((precision*recall)/(2**2*precision+recall+K.epsilon()))

def multi_rle_encode(img):
    labels = label(img[:, :, 0])
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

extra = {
    'bce_jaccard_loss': bce_jaccard_loss,
    'f2_micro': f2_micro
}
ship_dir = '../input/airbus-ship-detection/'
test_image_dir = os.path.join(ship_dir, 'test_v2')
test_paths = os.listdir(test_image_dir)


Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/classification_models/resnext/__init__.py:4: UserWarning: Current ResNext models are deprecated, use keras.applications ResNeXt models
  warnings.warn('Current ResNext models are deprecated, '


In [4]:
from tqdm import tqdm_notebook
from skimage.morphology import binary_opening, disk
from skimage.morphology import label

unet = load_model("../input/v1airbusmodels/unet_weights.hdf5", custom_objects=extra)
pspnet = load_model("../input/v1airbusmodels/pspnet_weights.hdf5", custom_objects=extra)
linknet = load_model("../input/v1airbusmodels/linknet_weights.hdf5", custom_objects=extra)
fpn = load_model("../input/v1airbusmodels/fpn_weights.hdf5", custom_objects=extra)
models = [unet, pspnet, linknet, fpn]

# unet_rows = []
# pspnet_rows = []
# linknet_rows = []
# fpn_rows = []
out_pred_rows = [[],[],[],[]]
for c_img_name in tqdm_notebook(test_paths):
    c_path = os.path.join(test_image_dir, c_img_name)
    c_img = imread(c_path)
    c_img = np.expand_dims(c_img, 0)/255.0
    for m in range(len(models)):
        cur_seg = None
        cur_seg = models[m].predict(c_img)[0]
        cur_seg = binary_opening(cur_seg>0.5, np.expand_dims(disk(2), -1))
        cur_rles = multi_rle_encode(cur_seg)
        if len(cur_rles)>0:
            for c_rle in cur_rles:
                out_pred_rows[m] += [{'ImageId': c_img_name, 'EncodedPixels': c_rle}]
        else:
            out_pred_rows[m] += [{'ImageId': c_img_name, 'EncodedPixels': None}]
        gc.collect()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


ValueError: Error when checking input: expected input_1 to have shape (528, 528, 3) but got array with shape (768, 768, 3)

In [5]:
submission_df1 = pd.DataFrame(out_pred_rows[0])[['ImageId', 'EncodedPixels']]
submission_df1.to_csv('unet.csv', index=False)
submission_df1.sample(3)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [6]:
submission_df2 = pd.DataFrame(out_pred_rows[1])[['ImageId', 'EncodedPixels']]
submission_df2.to_csv('pspnet.csv', index=False)
submission_df2.sample(3)

KeyError: "['ImageId' 'EncodedPixels'] not in index"

In [7]:
submission_df3 = pd.DataFrame(out_pred_rows[2])[['ImageId', 'EncodedPixels']]
submission_df3.to_csv('linknet.csv', index=False)
submission_df3.sample(3)

KeyError: "['ImageId' 'EncodedPixels'] not in index"

In [8]:
submission_df4 = pd.DataFrame(out_pred_rows[3])[['ImageId', 'EncodedPixels']]
submission_df4.to_csv('fpn.csv', index=False)
submission_df4.sample(3)

KeyError: "['ImageId' 'EncodedPixels'] not in index"